### Задание:

1. Выберите набор данных (датасет) для решения задачи классификации или регресии.
2. В случае необходимости проведите удаление или заполнение пропусков и кодирование категориальных признаков.
3. С использованием метода train_test_split разделите выборку на обучающую и тестовую.
4. Обучите 1) одну из линейных моделей, 2) SVM и 3) дерево решений. Оцените качество моделей с помощью трех подходящих для задачи метрик. Сравните качество полученных моделей.
5. Произведите для каждой модели подбор одного гиперпараметра с использованием GridSearchCV и кросс-валидации.
6. Повторите пункт 4 для найденных оптимальных значений гиперпараметров. Сравните качество полученных моделей с качеством моделей, полученных в пункте 4.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier 
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
%matplotlib inline 
sns.set(style="ticks")

### 1. Выберите набор данных (датасет) для решения задачи классификации или регресии.

In [2]:
# загружаем датасет
data=pd.read_csv('DOW30.csv', sep=",")

In [3]:
# размер набора данных
data.shape

(2448, 14)

In [4]:
# типы колонок
data.dtypes

Date             object
LABEL             int64
Open            float64
High            float64
Low             float64
Close           float64
Volume          float64
InterestRate    float64
ExchangeRate    float64
VIX             float64
Gold            float64
Oil             float64
TEDSpread       float64
EFFR            float64
dtype: object

In [5]:
# проверим есть ли пропущенные значения
data.isnull().sum()

Date            0
LABEL           0
Open            0
High            0
Low             0
Close           0
Volume          0
InterestRate    0
ExchangeRate    0
VIX             0
Gold            0
Oil             0
TEDSpread       0
EFFR            0
dtype: int64

In [6]:
data.head()

,Date,LABEL,Open,High,Low,Close,Volume,InterestRate,ExchangeRate,VIX,Gold,Oil,TEDSpread,EFFR
0,2008/4/1,0,12266.63965,12659.82031,12266.46973,12654.36035,295530000.0,1.77,1.5615,22.68,897.00,100.92,1.30,2.38
1,2008/4/2,1,12651.66992,12696.29004,12555.16992,12608.91992,232760000.0,1.72,1.5618,23.43,893.50,104.83,1.31,2.18
2,2008/4/3,0,12605.83008,12675.12012,12527.75000,12626.03027,183870000.0,1.70,1.5667,23.21,898.25,103.92,1.35,2.19
3,2008/4/4,1,12626.03027,12688.48047,12528.16016,12609.41992,181260000.0,1.67,1.5735,22.45,905.25,106.09,1.40,2.26
4,2008/4/7,0,12612.58984,12733.66016,12583.28027,12612.42969,198070000.0,1.62,1.5713,22.42,914.70,108.91,1.28,2.24


In [7]:
# Основные статистические характеристки набора данных
data.describe()

,LABEL,Open,High,Low,Close,Volume,InterestRate,ExchangeRate,VIX,Gold,Oil,TEDSpread,EFFR
count,2448.000000,2448.000000,2448.000000,2448.000000,2448.000000,2.448000e+03,2448.000000,2448.000000,2448.000000,2448.000000,2448.000000,2448.000000,2448.000000
mean,0.542484,14776.818372,14858.314373,14691.128288,14779.709081,1.860912e+08,1.758832,1.281169,19.887296,1274.313991,75.563137,0.415776,0.382831
std,0.498294,4223.631326,4220.319281,4228.302892,4224.400625,1.104695e+08,0.623806,0.130193,9.988958,241.554767,24.673508,0.457680,0.516170
min,0.000000,6547.009766,6709.609863,6469.950195,6547.049805,8.410000e+06,0.470000,1.037500,9.140000,692.500000,26.190000,0.090000,0.040000
25%,0.000000,11406.242188,11479.565183,11301.890380,11408.205080,1.032550e+08,1.310000,1.157950,13.547500,1144.875000,50.805000,0.210000,0.110000
50%,1.000000,14548.929690,14586.964845,14464.049805,14555.000000,1.513600e+08,1.715000,1.301650,16.920000,1258.425000,78.495000,0.270000,0.150000
75%,1.000000,17724.763185,17797.615230,17629.057132,17723.240235,2.406975e+08,2.330000,1.367925,22.510000,1374.500000,96.170000,0.400000,0.370000
max,1.000000,26584.279300,26616.710940,26435.339840,26616.710940,8.239400e+08,2.910000,1.601000,80.860000,1891.000000,145.310000,4.580000,2.970000


### 2. В случае необходимости проведите удаление или заполнение пропусков и кодирование категориальных признаков.

In [65]:
le=LabelEncoder()
le.fit(data.Date)
data['Date']=le.transform(data.Date)

In [66]:
# типы колонок
data.dtypes

Date              int32
LABEL             int64
Open            float64
High            float64
Low             float64
Close           float64
Volume          float64
InterestRate    float64
ExchangeRate    float64
VIX             float64
Gold            float64
Oil             float64
TEDSpread       float64
EFFR            float64
dtype: object

### 3. С использованием метода train_test_split разделите выборку на обучающую и тестовую.

In [67]:
# Разделим выборку на обучающую и тестовую
# X_train - обучающая выборка (матрица объект-признак)
# y_train - обучающая выборка (вектор целевого признака)
# X_test - тестовая выборка (матрица объект-признак)
# y_test - тестовая выборка (вектор целевого признака)

X_train, X_test, y_train, y_test = train_test_split(
    data, data['LABEL'], test_size= 0.5, random_state= 1)

In [68]:
# Размер обучающей выборки
X_train.shape, y_train.shape

((1224, 14), (1224,))

In [69]:
# Размер тестовой выборки
X_test.shape, y_test.shape

((1224, 14), (1224,))

### 4. Обучите 1) одну из линейных моделей, 2) SVM и 3) дерево решений. Оцените качество моделей с помощью трех подходящих для задачи метрик. Сравните качество полученных моделей.

In [70]:
# обучение линейной модели
sgd = SGDClassifier().fit(X_train, y_train)

C:\foruniver\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


In [71]:
age_sgd = sgd.predict(X_test)

In [72]:
# метрики линейной модели
accuracy_score(y_test, age_sgd), \
precision_score(y_test, age_sgd), \
recall_score(y_test, age_sgd)

(0.5343137254901961, 0.5343137254901961, 1.0)

In [73]:
# обучение SVC
svc = SVC(gamma='auto').fit(X_train, y_train)

In [74]:
age_svc = svc.predict(X_test)

In [75]:
# метрики SVC
accuracy_score(y_test, age_svc), \
precision_score(y_test, age_svc), \
recall_score(y_test, age_svc)

(0.5343137254901961, 0.5343137254901961, 1.0)

In [76]:
# обучение дерева решений
tree = DecisionTreeClassifier(random_state=1, max_depth=0.75).fit(X_train, y_train)

In [77]:
age_tree = tree.predict(X_test)

In [78]:
# метрики дерева решений
accuracy_score(y_test, age_tree), \
precision_score(y_test, age_tree), \
recall_score(y_test, age_tree)

(0.5343137254901961, 0.5343137254901961, 1.0)

### 5. Произведите для каждой модели подбор одного гиперпараметра с использованием GridSearchCV и кросс-валидации.

In [79]:
# линейная модель
scores_sgd = cross_val_score(SGDClassifier(), 
                         X_train, y_train, cv=2)
scores_sgd

C:\foruniver\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
C:\foruniver\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


array([0.44934641, 0.44934641])

In [80]:
# SVC
scores_svm_svc = cross_val_score(SVC(gamma='auto'), 
                         X_train, y_train, cv=2)
scores_svm_svc

array([0.55065359, 0.55065359])

In [81]:
# дерево решений
scores_decision_tree = cross_val_score(DecisionTreeClassifier(), 
                         X_train, y_train, cv=2)
scores_decision_tree

array([1., 1.])

In [82]:
parameters = {'alpha':[0.5,0.4,0.3,0.2,0.1]}
clf_gs_sgd = GridSearchCV(SGDClassifier(), parameters, cv=2, scoring='accuracy')
clf_gs_sgd.fit(X_train, y_train)

C:\foruniver\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
C:\foruniver\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
C:\foruniver\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and 

GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'alpha': [0.5, 0.4, 0.3, 0.2, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [83]:
# Лучшее значение параметров для линейной модели
clf_gs_sgd.best_params_

{'alpha': 0.2}

In [84]:
parameters = {'gamma':[0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1]}
clf_gs_svm_svc = GridSearchCV(SVC(), parameters, cv=2, scoring='accuracy')
clf_gs_svm_svc.fit(X_train, y_train)

GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'gamma': [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [85]:
# Лучшее значение параметров для SVC

clf_gs_svm_svc.best_params_

{'gamma': 0.9}

In [86]:
parameters = {'min_impurity_decrease':[0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1]}
clf_gs_decision_tree = GridSearchCV(DecisionTreeClassifier(), parameters, cv=2, scoring='accuracy')
clf_gs_decision_tree.fit(X_train, y_train)

GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'min_impurity_decrease': [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [87]:
# Лучшее значение параметров для дерева решений
clf_gs_decision_tree.best_params_

{'min_impurity_decrease': 0.4}

### 6. Повторите пункт 4 для найденных оптимальных значений гиперпараметров. Сравните качество полученных моделей с качеством моделей, полученных в пункте 4.

In [96]:
# обучение линейной модели
sgd2 = SGDClassifier(alpha=0.2).fit(X_train, y_train)

C:\foruniver\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


In [97]:
age_sgd2 = sgd2.predict(X_test)

In [98]:
# метрики линейной модели
accuracy_score(y_test, age_sgd2), \
precision_score(y_test, age_sgd2), \
recall_score(y_test, age_sgd2)

C:\foruniver\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


(0.46568627450980393, 0.0, 0.0)

In [99]:
# обучение SVC
svc2 = SVC(gamma=0.9).fit(X_train, y_train)

In [100]:
age_svc2 = svc.predict(X_test)

In [101]:
# метрики SVC
accuracy_score(y_test, age_svc2), \
precision_score(y_test, age_svc2), \
recall_score(y_test, age_svc2)

(0.5343137254901961, 0.5343137254901961, 1.0)

In [102]:
# обучение дерева решений
tree2 = DecisionTreeClassifier(random_state=1, 
                               min_impurity_decrease=0.4, 
                               max_depth=0.75).fit(X_train, y_train)

In [103]:
age_tree2 = tree.predict(X_test)

In [104]:
# метрики дерева решений
accuracy_score(y_test, age_tree2), \
precision_score(y_test, age_tree2), \
recall_score(y_test, age_tree2)

(0.5343137254901961, 0.5343137254901961, 1.0)

метрики: accuracy, precision, recall
    
в пункте 4:

- метрики линейной модели
- (0.5343137254901961, 0.5343137254901961, 1.0)
- метрики SVC
- (0.5343137254901961, 0.5343137254901961, 1.0)
- метрики дерева решений
- (0.5343137254901961, 0.5343137254901961, 1.0)

в пункте 6:
- метрики линейной модели
- (0.46568627450980393, 0.0, 0.0)
- метрики SVC
- (0.5343137254901961, 0.5343137254901961, 1.0)
- метрики дерева решений
- (0.5343137254901961, 0.5343137254901961, 1.0)